In [1]:
import os
import argparse
import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from torchdyn.core import NeuralODE
from torchdyn.nn import DataControl, DepthCat, Augmenter, GalLinear, Fourier
import matplotlib.pyplot as plt
import torch.utils.data as data

In [2]:
from torchdyn.utils import *

In [3]:
device=torch.device('cpu')

In [4]:
class TraceDataset(Dataset):
    """ Dataset for loading traces"""
    def __init__(self,csv_file):
        self.tracedf=pd.read_csv(csv_file)
        #self.tracedf=self.tracedf.tail(200)
        self.tracedf=self.tracedf.apply(lambda x: x*500000)
        #self.tracedf['Subsystem #1|CTLs|Vars|Igrid_d'] = self.tracedf['Subsystem #1|CTLs|Vars|Igrid_d'].apply(lambda x: x*1000)
        tempx=self.tracedf[["Id2","Iq2","Vd2","Vq2","VBat2ds","VBat2qs","V_mg_d","V_mg_q","I_mg_d","I_mg_q"]]
        #tempx=self.tracedf[["I_mg_d","I_mg_q"]]
        self.x=torch.tensor(tempx.values).float().to(device)
        #self.x=nn.functional.normalize(self.x)
        self.y=torch.tensor(self.tracedf[["I_mg_d","I_mg_q"]].values).float().to(device)
        #self.y=nn.functional.normalize(self.y)
        self.t=torch.tensor(np.linspace(0,30,len(self.tracedf))).float().squeeze().to(device)
        
    def __len__(self):
        return len(self.tracedf)
    
    def __getitem__(self,idx):
        
        #tempx=torch.from_numpy(self.x.iloc[[idx]].to_numpy()).float()
        #tempx=tempx.squeeze()
        #tempy=torch.from_numpy(self.y.iloc[[idx]].to_numpy()).float()
        #tempy=tempy.squeeze()
        #tempt=torch.from_numpy(self.t.iloc[[idx]].to_numpy()).float()
        sample={'x':self.x[idx,:],'y':self.y[idx,:],'t':self.t[idx]}
        return sample

In [5]:
td=TraceDataset('Trace_single_30min.csv')

In [6]:
X_train = td.x
y_train = td.y
train = data.TensorDataset(X_train, y_train)
trainloader = data.DataLoader(train, batch_size=500, shuffle=True)

In [7]:
import pytorch_lightning as pl

In [8]:
class Learner(pl.LightningModule):
    def __init__(self, t_span:torch.Tensor, model:nn.Module):
        super().__init__()
        self.model, self.t_span = model, t_span
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch      
        t_eval, y_hat = self.model(x, self.t_span)
        y_hat = y_hat[-1] # select last point of solution trajectory
        loss = nn.CrossEntropyLoss()(y_hat, y)
        return {'loss': loss}   
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.01)

    def train_dataloader(self):
        return trainloader

In [9]:
f = nn.Sequential(DataControl(),
                  nn.Linear(8+2, 64),
                  nn.Tanh(),
                  nn.Linear(64, 2))

t_span = td.t

# Neural ODE
model = NeuralODE(f, sensitivity='adjoint', solver='tsit5', atol=1e-3, rtol=1e-3).to(device)

Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.


In [ ]:
learn = Learner(t_span, model)

trainer = pl.Trainer(min_epochs=1, max_epochs=1)
trainer.fit(learn)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\amol\.conda\envs\ode\lib\site-packages\pytorch_lightning\trainer\trainer.py:1814: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
Missing logger folder: C:\Users\amol\Documents\powersys\neuralOde\lightning_logs

  | Name  | Type      | Params
------------------------------------
0 | model | NeuralODE | 322   
------------------------------------
322       Trainable params
0         Non-trainable params
322       Total params
0.001     Total estimated model params size (MB)
C:\Users\amol\.conda\envs\ode\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` a

Training: 0it [00:00, ?it/s]

In [ ]:
temp=next(iter(trainloader))

In [ ]:
len(temp)

In [ ]:
temp[0].shape

In [ ]:
temp[1].shape